# Tests Numpy

## Format JSON

In [6]:
import numpy as np
from test_numpy import to_json_tab, to_json, ndarray
from cfield import Cutil
from dataset import Sdataset
import ntv_pandas as npd

a = np.arange(1,25).reshape((2,3,2,2))
print(to_json(a))

axes = {'x': ['x1', 'x2'], 
        'y': ['y1', 'y2', 'y3'], 
        'z': ['z1', 'z2'], 
        'option': [True, False]}
js = to_json_tab(a, axes)

print(js)
nt = Sdataset.ntv(js)

print(nt.to_ntv())
print(npd.read_json(js))
print(ndarray(js))

[[[[1, 2], [3, 4]], [[5, 6], [7, 8]], [[9, 10], [11, 12]]], [[[13, 14], [15, 16]], [[17, 18], [19, 20]], [[21, 22], [23, 24]]]]
{'x': [['x1', 'x2'], [12]], 'y': [['y1', 'y2', 'y3'], [4]], 'z': [['z1', 'z2'], [2]], 'option': [[True, False], [1]], 'val': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]}
{"x": [["x1", "x2"], [12]], "y": [["y1", "y2", "y3"], [4]], "z": [["z1", "z2"], [2]], "option": [[true, false], [1]], "val": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]}
     x   y   z option  val
0   x1  y1  z1   True    1
1   x1  y1  z1  False    2
2   x1  y1  z2   True    3
3   x1  y1  z2  False    4
4   x1  y2  z1   True    5
5   x1  y2  z1  False    6
6   x1  y2  z2   True    7
7   x1  y2  z2  False    8
8   x1  y3  z1   True    9
9   x1  y3  z1  False   10
10  x1  y3  z2   True   11
11  x1  y3  z2  False   12
12  x2  y1  z1   True   13
13  x2  y1  z1  False   14
14  x2  y1  z2   True   15
15  x2  y1  